# Rooflines 
> Plotting rooflines with Altair

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]

In [ ]:
#hide
import numpy as np
import pandas as pd

import altair as alt

In [1]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
tableOverviewExperiments(['data/overview_experiments_imagenet.csv'])

C:\Users\alinav\Documents\GitHub\Qutibench_Web\_notebooks\scripts\altair_plots.py:276: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
C:\Users\alinav\Documents\GitHub\Qutibench_Web\_notebooks\scripts\altair_plots.py:272: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


# Rooflines for all hardware platforms

In [ ]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load dataset and plot it
rooflines(pd.read_csv("data/processed_csv/rooflines_hardware_neural_networks.csv"), 'All')

# Performance Predictions for IMAGENET, MNIST and CIFAR

In [ ]:
#hide_input
%run scripts/altair_plots.py  #run the heatmaps script
#load dataset and plot it
heatmap(pd.read_csv("data/processed_csv/performance_prediction_imagenet_mnist_cifar10.csv"), 
        'red', 
        'Performance Predictions for IMAGENET, MNIST and CIFAR10')